In [157]:
from pymongo import MongoClient 
import pandas as pd 
import json
from pymongo import errors 
from django.core.validators import URLValidator
from django.core.exceptions import ValidationError
from pymongo.errors import BulkWriteError

class IndeedMongodbDao:
    def __init__(self):
        self.conn = MongoClient() 
        self.db = self.conn.Indeed
        self.collection = self.db.data
        
    def _valid_url_format(self,url):
        val = URLValidator()
        try:
            val(url)
        except ValidationError as e:
            raise Exception('bad format for url {}'.format(ur))
    
    def insert_data_bulk(self,data):
        try:
            self.collection.insert_many(data)
        except BulkWriteError as bwe:
            print(bwe.details)
            #you can also take this component and do more analysis
            print(bwe.details['writeErrors'])
            raise
    
    def insert_data(self, url, title, name, address, publication_date,salaire, description, localisation):
        
        try:
            if url == "":
                raise Exception('url cannot be empty {}'.format(ur))

            self._valid_url_format(url)

            if title == "":
                raise Exception('title cannot be empty {}'.format(title))

            if name == "":
                raise Exception('the name of company cannot be be empty {}'.format(title))

            if description == "":
                raise Exception('description of company cannot be be empty {}'.format(title))

            #URL,Titre,Nom entreprise,Adresse,Date de publication,salaire,description,localisation
            line_to_insert = {
                                "url": url,
                                "titre":title,
                                "nom_entreprise":name,
                                "adresse":address,
                                "date_de_publication":publication_date,
                                "salaire":salaire,
                                "description":description,
                                "localisation":localisation
                             }

            # Insert Data 
            result = self.collection.insert_one(line_to_insert) 
        except Exception as e:
            print(e)
    
    def get_all_data(self):
        data = self.collection.find({})
        return data
    
    def description_exist(self, description):
        return self.collection.find({"description" : description}).count() > 0
        
    def url_exist(self, url):
        return self.collection.find({"url" : url})

    
class Csv_MongodbMigratorTool:
    
    def __init__(self):
        self.dataset = pd.read_csv("archiv_to_mongo/indeed.part.1.csv")
        self.mongodbdao = IndeedMongodbDao()
        
    def bulk_migrate(self):
        records = json.loads(self.dataset.T.to_json()).values()
        self.mongodbdao.insert_data_bulk(records)
    
    def migrate(self):
        for index, row in self.dataset.iterrows():
            if self.mongodbdao.description_exist(row["description"]) == False:
                self.mongodbdao.insert_data(row["URL"],row["Titre"],row["Nom entreprise"],row["Adresse"],row["Date de publication"],row["salaire"], row["description"], row["localisation"])
                        
    

In [158]:
tool = Csv_MongodbMigratorTool()
tool.migrate()

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:70: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.


In [65]:
df = pd.read_csv("archiv_to_mongo/indeed.part.2.csv")
for index, row in df.iteritems():
    print(index)

#df.duplicated(subset=["URL"], keep='first')
#URL = df["URL"]
#df[URL.isin(URL[URL.duplicated()])]

URL
Titre
Nom entreprise
Adresse
Date de publication
salaire
description
localisation
